In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [12]:
ratings_df = pd.read_csv('data/wa_ratings_data.csv')
ratings_df = ratings_df.rename(columns={'Unnamed: 0':'Username'})
ratings_df = ratings_df.set_index('Username')
ratings_df.drop('Unnamed: 1', axis=1, inplace=True)

In [10]:
ratings_df

,!dentity Crisis?,"""In Pursuit of Par"" Par 72 Edition","""Oh My God! There's An Axe In My Head."" The Game of International Diplomacy","""Tarleton's Quarter!""",$ue Me!,& Cetera,'65: Squad-Level Combat in the Jungles of Vietnam,"'Ere We Go: Orks in Warhammer 40,000",(Target Bookshelf Edition) Risk (aka Risk),(Target Bookshelf Edition) Scrabble (aka Scrabble),...,Конфетти,Містеріум,…aber bitte mit Sahne (aka Piece o' Cake),エアライン・ヨーロッパ,ゲシェンク,メディチ カードゲーム,三千世界の鴉,大卫与歌利亚 (大衛與歌利亞),妖精奇譚,小市場
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
-E..,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12thManStanding,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2 Salt City Gamers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
253bri,NaN,NaN,NaN,NaN,NaN,9.9,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2BABeachBum,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2CoF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3M4FUN4U2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45pole,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5th_Para_Bde,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
